In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd

# Cargar los DataFrames desde los archivos CSV previamente guardados
df = '/content/drive/My Drive/PI MLOps - STEAM/Total.csv'

In [ ]:
archivo_csv = ('Total.csv')
df = pd.read_csv(df)

In [ ]:
df['genres'] = df['genres'].apply(lambda x: x.replace("'", "").strip("[]"))

In [ ]:
df1=df[['genres','release_date','playtime_forever']]

In [ ]:
# #app = FastAPI()

# @app.get("/AnioConMasHorasJugadas")
def PlayTimeGenre(genero: str):

    """
    Se obtiene el año de lanzamiento con más horas jugadas para un género dado.

    Argumento:
        genero (str): El género de los juegos (no es sensible a mayúsculas o minúsculas).

    Returns:
        dict: Un diccionario que contiene el año de lanzamiento con más horas jugadas.
              Ejemplo: {"Año de lanzamiento con más horas jugadas para [genero]": [año]}
    """

    # Convierto el género ingresado por el usuario a minúsculas para que lo encuentre
    genero = genero.lower()

    # Filtro el df por el género deseado (en minúsculas)
    df_genero = df1[df1['genres'].str.lower() == genero]

    # Verifico si hay o no registros
    if df_genero.empty:
        return {"Mensaje": "No se encontraron registros para el género especificado."}

    # Hago un agrupamiento por el release_date (año de lanz.) y que se sumen las horas jugadas para cada año
    gen_agrup = df_genero.groupby('release_date')['playtime_forever'].sum().reset_index()

    # Encuentro el año en el que mas tiempo se jugo
    max_year = gen_agrup.loc[gen_agrup['playtime_forever'].idxmax()]

    # Convierto playtime_forever de segundos a horas, con dos deciamles
    horas_jugadas = round(max_year['playtime_forever'] / 60 / 60, 2)

    # Le doy forma de diccionario al resultado
    resultado = {"Año de lanzamiento con más horas jugadas para " + genero.capitalize(): int(max_year['release_date'])}

    return resultado

genero_deseado = input("Ingrese el género: ")
res = PlayTimeGenre(genero_deseado)
print(res)

Ingrese el género: action
{'Año de lanzamiento con más horas jugadas para Action': 1998}


In [ ]:
ruta_guardar = '/content/drive/My Drive/PI MLOps - STEAM/df1.csv'
df1.to_csv(ruta_guardar, index=False)

In [ ]:
df2 = df[['genres', 'release_date', 'playtime_forever', 'user_id']]

In [ ]:
# @app.get("/UsuarioConMasHorasJugadas")
def UserForGenre(genero: str):
    """
    Se obtiene el usuario con más horas jugadas y la acumulación de las mismas por año,
    para un género dado.

    Argumento:
        genero (str): El género de los juegos (no es sensible a mayúsculas o minúsculas).

    Returns:
        dict: Un diccionario que contiene el usuario con más horas jugadas y la acumulación
        de horas para cada año
              Ejemplo: {"Usuario con más horas jugadas para [genero]": [usuario]}
                        "Acumulación de horas jugadas por año para [genero]": [año: horas]
    """



    # Convierto lo ingresado a minúsculas
    genero = genero.lower()

    # Filtro el df por el género ingresado
    df_genero = df2[df2['genres'].str.lower() == genero]

    # Verifico si se encontraron registros para el género
    if df_genero.empty:
        return {"Mensaje": "No se encontraron registros para el género especificado."}

    # Agrupo por usuario sumando las horas que ha jugado en ese género
    usuario_horas = df_genero.groupby('user_id')['playtime_forever'].sum().reset_index()

    # Encuentro el usuario que más horas jugó
    usuario_maximo = usuario_horas.loc[usuario_horas['playtime_forever'].idxmax()]

    # Agrupo por año de lanzamiento y a eso le sumo las horas que jugó en ese año y de ese género
    año_horas = df_genero.groupby('release_date')['playtime_forever'].sum().reset_index()

    # Renombrar las columnas para que tengan nombres más amigables
    año_horas.rename(columns={'release_date': 'año', 'playtime_forever': 'horas_jugadas'}, inplace=True)

    # Convertir playtime_forever de segundos a horas y redondear a dos decimales
    año_horas['horas_jugadas'] = round(año_horas['horas_jugadas'] / 60 / 60, 2)

    # Formatear el resultado como un diccionario
    result = {
        "Usuario con más horas jugadas para " + genero.capitalize(): usuario_maximo['user_id'],
        "Acumulación de horas jugadas por año para " + genero.capitalize(): año_horas.astype({'año': int}).to_dict(orient='records')
    }

    return result

genero = input("Ingrese el género: ")
resultado = UserForGenre(genero)
print(resultado)


Ingrese el género: action
{'Usuario con más horas jugadas para Action': 'lasati', 'Acumulación de horas jugadas por año para Action': [{'año': 1993, 'horas_jugadas': 65.48}, {'año': 1994, 'horas_jugadas': 58.73}, {'año': 1995, 'horas_jugadas': 8.2}, {'año': 1996, 'horas_jugadas': 35.73}, {'año': 1997, 'horas_jugadas': 0.91}, {'año': 1998, 'horas_jugadas': 138.14}, {'año': 1999, 'horas_jugadas': 42.08}, {'año': 2000, 'horas_jugadas': 16.01}, {'año': 2001, 'horas_jugadas': 0.01}, {'año': 2002, 'horas_jugadas': 0.14}, {'año': 2003, 'horas_jugadas': 30.77}, {'año': 2004, 'horas_jugadas': 57.18}, {'año': 2005, 'horas_jugadas': 58.5}, {'año': 2006, 'horas_jugadas': 22.38}, {'año': 2007, 'horas_jugadas': 37.63}, {'año': 2008, 'horas_jugadas': 10.89}, {'año': 2009, 'horas_jugadas': 24.8}, {'año': 2010, 'horas_jugadas': 46.06}, {'año': 2011, 'horas_jugadas': 64.88}, {'año': 2012, 'horas_jugadas': 134.48}, {'año': 2013, 'horas_jugadas': 34.37}, {'año': 2014, 'horas_jugadas': 6.83}, {'año': 2015,

In [ ]:
ruta_guardar = '/content/drive/My Drive/PI MLOps - STEAM/df2.csv'
df2.to_csv(ruta_guardar, index=False)  # index=False evita que se guarde el índice del DataFrame en el archivo

In [ ]:
df3= df[['release_date','item_name','sentiment_analysis','recommend']]

#@app.get("/MasRecomendados")
def UsersRecommend(anio: int):
    """
    Devuelve el top 3 de juegos MÁS recomendados por usuarios para el año dado

    Argumento:
        año (int)

    Returns:
        dict: Un diccionario que contiene los tres juegos mas recomendados y con mas reseñas positivas
        Ejemplo: {"Puesto 1": 'nombre de juego', "Puesto 2": 'nombre de juego', "Puesto 3": 'nombre de juego'}
    """

    # Filtro el df por año
    df_year = df3[df3['release_date'] == anio]

    # Verifico si se encontraron registros
    if df_year.empty:
        return {"Mensaje": "No se encontraron registros para el año especificado."}

    # Filtro juegos recomendados
    df_recommended = df_year[(df_year['recommend'] == True) & (df_year['sentiment_analysis'] >= 1)]

    # Hago el conteo de recomendaciones por juego
    game_recommend_counts = df_recommended['item_name'].value_counts()

    # Tomo los 3 juegos más recomendados
    top_3_games = game_recommend_counts.head(3).index.tolist()

    # Formateo el resultado como un diccionario
    result = {"Puesto " + str(i + 1): juego for i, juego in enumerate(top_3_games)}

    return result

anio_deseado = int(input("Ingrese el año: "))
resultado = UsersRecommend(anio_deseado)
print(resultado)



Ingrese el año: 2013
{'Puesto 1': 'Left 4 Dead 2', 'Puesto 2': "Garry's Mod", 'Puesto 3': 'Robocraft'}


In [ ]:
ruta_guardar = '/content/drive/My Drive/PI MLOps - STEAM/df3.csv'
df3.to_csv(ruta_guardar, index=False)  # index=False evita que se guarde el índice del DataFrame en el archivo

In [ ]:
df5= df[['release_date','sentiment_analysis']]


 # @app.get("/SentReseñas")
def sentiment_analysis(anio:int):

    """
    Según el año de lanzamiento, se devuelve una lista con la cantidad de registros de reseñas de
    usuarios que se encuentren categorizados con un análisis de sentimiento.

    Argumento:
        año (int)

    Returns:
        dict: Un diccionario que contiene las tres categorias de reseñas y sus cantidades
        Ejemplo: {Negative = [cantidad_reseñas], Neutral = [cantidad_reseñas], Positive = [cantidad_reseñas]}

    """

    # Filtrar el DataFrame por el año ingresado
    df_año = df5[df5['release_date'] == anio]

    # # Verificar si se encontraron registros para el año
    if df_año.empty:
        return {"Mensaje": "No se encontraron registros para el año especificado."}

    # # Contar la cantidad de registros para cada categoría de análisis de sentimiento
    sentiment_analysis_column = df_año['sentiment_analysis']
    sentiment = sentiment_analysis_column.value_counts().to_dict()

    # Crear un diccionario con los resultados
    result = {
        "Negative": sentiment.get(0, 0),
        "Neutral": sentiment.get(1, 0),
        "Positive": sentiment.get(2, 0)
    }

    return result

anio_deseado = int(input("Ingrese el año: "))
resultado = sentiment_analysis(anio_deseado)
print(resultado)


Ingrese el año: 2013
{'Negative': 132, 'Neutral': 141, 'Positive': 425}


In [ ]:
ruta_guardar = '/content/drive/My Drive/PI MLOps - STEAM/df5.csv'
df5.to_csv(ruta_guardar, index=False)  # index=False evita que se guarde el índice del DataFrame en el archivo